# Probabilities and k-Means Clustering

Using the IMDB data, construct a feature matrix, and apply `k-Means` to the data to extract clusters. 

We then inspect various aspects of probability associated with these clusterings.

In [5]:
import json

import pandas as pd
import numpy as np

In [6]:
actor_name_map = {}
movie_actor_map = {}
actor_genre_map = {}


with open("imdb_movies_2000to2022.prolific.json", "r") as in_file:
    for line in in_file:
        
        # Read the movie on this line and parse its json
        this_movie = json.loads(line)
                    
        # Add all actors to the id->name map
        for actor_id,actor_name in this_movie['actors']:
            actor_name_map[actor_id] = actor_name
            
        # For each actor, add this movie's genres to that actor's list
        for actor_id,actor_name in this_movie['actors']:
            this_actors_genres = actor_genre_map.get(actor_id, {})
            
            # Increment the count of genres for this actor
            for g in this_movie["genres"]:
                this_actors_genres[g] = this_actors_genres.get(g, 0) + 1
                
            # Update the map
            actor_genre_map[actor_id] = this_actors_genres
            
        # Finished with this film
        movie_actor_map[this_movie["imdb_id"]] = ({
            "movie": this_movie["title"],
            "actors": set([item[0] for item in this_movie['actors']]),
            "genres": this_movie["genres"]
        })

In [7]:
print("Known Actors:", len(actor_name_map))
print("Known Movies:", len(movie_actor_map))

Known Actors: 33609
Known Movies: 20620


## Read CSV of Movies to Cluster IDs

Using the provided movie-to-cluster mapping CSV file, we assess the distributions of movies per cluster and ask questions about genres and actors in each cluster.

In [8]:
cluster_df = pd.read_csv("movie_to_cluster.csv")

In [9]:
cluster_df["cluster"].value_counts()

cluster
6     3177
0     3097
15    1754
13    1705
2     1503
12    1466
1     1376
3     1240
14     893
8      774
10     761
4      655
11     640
7      635
5      560
9      384
Name: count, dtype: int64

In [10]:
cluster_pr_map = {cluster_id:cluster_pr for cluster_id,cluster_pr in (cluster_df["cluster"].value_counts() / cluster_df.shape[0]).items()}
cluster_pr_df = pd.DataFrame(cluster_df["cluster"].value_counts() / cluster_df.shape[0])

cluster_pr_df

,count
cluster,
6,0.154074
0,0.150194
15,0.085063
13,0.082687
2,0.072890
12,0.071096
1,0.066731
3,0.060136
14,0.043307


## Assess Genre-Specific Cluster Probabilities

We want to determine, for a new movie with a known genre, to which cluster is it most likely to be assigned?

In [11]:
# For each genre, count the number of movies
genre_counts = {}

# For each movie, get its genres and update the genre count
for movie_id in movie_actor_map.keys():
    for genre in movie_actor_map[movie_id]["genres"]:
        genre_counts[genre] = genre_counts.get(genre, 0) + 1
        
genre_prs = []
for genre,g_count in genre_counts.items():
    genre_prs.append((genre, g_count/len(movie_actor_map)))
    
genre_prs_df = pd.DataFrame(genre_prs, columns=["genre", "probability"])
genre_pr_map = {row["genre"]:row["probability"] for idx,row in genre_prs_df.iterrows()}

genre_prs_df.sort_values(by="probability", ascending=False)

,genre,probability
5,Drama,0.492580
0,Comedy,0.291804
10,Thriller,0.194180
6,Action,0.181523
3,Horror,0.149224
8,Crime,0.134481
2,Romance,0.122260
7,Adventure,0.080844
9,Mystery,0.074442
4,Sci-Fi,0.051164


In [12]:
#Setting the genre we will be investigating
target_genre = "Sci-Fi"
target_genre1 = "Fantasy"
target_genre2 = "Adventure"


### Calculate Conditional Probabilities of the given Genre for each cluster

We want to calculate the conditional probability of a movie being assigned to a cluster given its genre. This probability can be calculated as follows:

$$ Pr(Y=Cluster <ID> | X=Genre) = \frac{Pr(X=Genre | Y=Cluster <ID>) Pr(Y=Cluster <ID>)}{Pr(X=Genre)}  $$

We have $Pr(X=Genre)$ above via `genre_prs_df`, and we have $Pr(Y=Cluster <ID>)$ above via `cluster_pr_df`, so all we need to calculate $Pr(X=Genre | Y=Cluster <ID>)$.


In [13]:
per_cluster_prs = []
for cluster_id,group in cluster_df.groupby("cluster"):

    this_cluster_genre_count = sum([
        1 if target_genre in movie_actor_map[m]["genres"] else 0 
        for m in group["movie_id"]
    ])
    
    total_genre_count = len([g for m in group["movie_id"] for g in movie_actor_map[m]["genres"]])
    
    # Calculate conditional probability of the target genre given this cluster
    ## Pr(X=Genre | Y=Cluster <ID>)
    p_cluster_given_genre = this_cluster_genre_count / total_genre_count
    print("Pr[%s| Cluster %02d]:" % (target_genre, cluster_id), "\t", p_cluster_given_genre)
    print("Pr[%s| Cluster %02d]:" % (target_genre1, cluster_id), "\t", p_cluster_given_genre)
    print("Pr[%s| Cluster %02d]:" % (target_genre2, cluster_id), "\t", p_cluster_given_genre)

    
    # Then add this calculated probability to `per_cluster_prs`,
    ## so we can track this conditional probability
    pr_genre_cluster = p_cluster_given_genre * group.shape[0] / cluster_df.shape[0]
    print("Pr[%s, Cluster %02d]:" % (target_genre, cluster_id), "\t", pr_genre_cluster)
    print("Pr[%s| Cluster %02d]:" % (target_genre1, cluster_id), "\t", p_cluster_given_genre)
    print("Pr[%s| Cluster %02d]:" % (target_genre2, cluster_id), "\t", p_cluster_given_genre)
    per_cluster_prs.append(pr_genre_cluster)


Pr[Sci-Fi| Cluster 00]: 	 0.0386402027027027
Pr[Fantasy| Cluster 00]: 	 0.0386402027027027
Pr[Adventure| Cluster 00]: 	 0.0386402027027027
Pr[Sci-Fi, Cluster 00]: 	 0.005803526080032506
Pr[Fantasy| Cluster 00]: 	 0.0386402027027027
Pr[Adventure| Cluster 00]: 	 0.0386402027027027
Pr[Sci-Fi| Cluster 01]: 	 0.01156224791610648
Pr[Fantasy| Cluster 01]: 	 0.01156224791610648
Pr[Adventure| Cluster 01]: 	 0.01156224791610648
Pr[Sci-Fi, Cluster 01]: 	 0.0007715641674375614
Pr[Fantasy| Cluster 01]: 	 0.01156224791610648
Pr[Adventure| Cluster 01]: 	 0.01156224791610648
Pr[Sci-Fi| Cluster 02]: 	 0.057951482479784364
Pr[Fantasy| Cluster 02]: 	 0.057951482479784364
Pr[Adventure| Cluster 02]: 	 0.057951482479784364
Pr[Sci-Fi, Cluster 02]: 	 0.004224106603642866
Pr[Fantasy| Cluster 02]: 	 0.057951482479784364
Pr[Adventure| Cluster 02]: 	 0.057951482479784364
Pr[Sci-Fi| Cluster 03]: 	 0.007735792918774174
Pr[Fantasy| Cluster 03]: 	 0.007735792918774174
Pr[Adventure| Cluster 03]: 	 0.007735792918774174

In [14]:
# For each cluster ID, calculate the posterior probability given the target genre
for cluster_id,cluster_genre_pr in enumerate(per_cluster_prs):

    # combine this cluster_genre_pr, cluster_pr_df, and genre_prs_df
    ## to calculate Pr(Cluster|Genre)
    pr_cluster_given_genre = cluster_genre_pr / genre_pr_map[target_genre]

    print("Pr[Cluster %02d | %s]:" % (cluster_id, target_genre), "\t", pr_cluster_given_genre)
    print("Pr[Cluster %02d | %s]:" % (cluster_id, target_genre1), "\t", pr_cluster_given_genre)
    print("Pr[Cluster %02d | %s]:" % (cluster_id, target_genre2), "\t", pr_cluster_given_genre)
    
    

Pr[Cluster 00 | Sci-Fi]: 	 0.11343005475855002
Pr[Cluster 00 | Fantasy]: 	 0.11343005475855002
Pr[Cluster 00 | Adventure]: 	 0.11343005475855002
Pr[Cluster 01 | Sci-Fi]: 	 0.015080239936078215
Pr[Cluster 01 | Fantasy]: 	 0.015080239936078215
Pr[Cluster 01 | Adventure]: 	 0.015080239936078215
Pr[Cluster 02 | Sci-Fi]: 	 0.0825602636655127
Pr[Cluster 02 | Fantasy]: 	 0.0825602636655127
Pr[Cluster 02 | Adventure]: 	 0.0825602636655127
Pr[Cluster 03 | Sci-Fi]: 	 0.00909230636898576
Pr[Cluster 03 | Fantasy]: 	 0.00909230636898576
Pr[Cluster 03 | Adventure]: 	 0.00909230636898576
Pr[Cluster 04 | Sci-Fi]: 	 0.019283290355989097
Pr[Cluster 04 | Fantasy]: 	 0.019283290355989097
Pr[Cluster 04 | Adventure]: 	 0.019283290355989097
Pr[Cluster 05 | Sci-Fi]: 	 0.0
Pr[Cluster 05 | Fantasy]: 	 0.0
Pr[Cluster 05 | Adventure]: 	 0.0
Pr[Cluster 06 | Sci-Fi]: 	 0.04592663925220342
Pr[Cluster 06 | Fantasy]: 	 0.04592663925220342
Pr[Cluster 06 | Adventure]: 	 0.04592663925220342
Pr[Cluster 07 | Sci-Fi]: 	 0.0

### Sample Titles in Each Cluster

We can use the above conditional probabilities to determine the most likely cluster given a movie genre.

Here, we sample movies in the most likely cluster to get a sense of what movies are in that cluster.

In [15]:
target_cluster = 0

In [16]:
for movie_id in cluster_df[cluster_df["cluster"] == target_cluster].sample(n=10, replace=False)["movie_id"]:
    this_movie = movie_actor_map[movie_id]
    print(movie_id, this_movie["movie"], this_movie["genres"])

tt4765284 Pitch Perfect 3 ['Comedy', 'Music']
tt2279233 Reunion 108 ['Comedy']
tt5564792 Stuck ['Comedy']
tt0348333 Waiting... ['Comedy']
tt7427356 The Last Laugh ['Comedy']
tt0357462 Bamboo Boys ['Comedy']
tt14212074 Billy Two Rivers ['Western']
tt4357714 The Sex Trip ['Comedy', 'Fantasy']
tt0388183 King's Ransom ['Comedy', 'Crime']
tt0119231 The Junior Defenders ['Comedy']


## Assess Actor-Specific Cluster Probabilities

Above, we determine the most likely cluster given a movie genre. Here, we ask the same question for a given actor.

In [17]:
# For each actor, count the number of movies
actor_counts = {}

# For each movie, get its genres and update the genre count
for movie_id in movie_actor_map.keys():
    for actor in movie_actor_map[movie_id]["actors"]:
        actor_counts[actor] = actor_counts.get(actor, 0) + 1
        
actor_prs = []
for actor,a_count in actor_counts.items():
    actor_prs.append((actor, a_count/len(movie_actor_map)))
    
actor_prs_df = pd.DataFrame(actor_prs, columns=["actor", "probability"])
actor_pr_map = {row["actor"]:row["probability"] for idx,row in actor_prs_df.iterrows()}

actor_prs_df.sort_values(by="probability", ascending=False)

,actor,probability
661,nm0000616,0.009408
260,nm0000514,0.004753
1032,nm0001744,0.004704
1204,nm0001803,0.004268
127,nm0222881,0.003637
...,...,...
15304,nm7580109,0.000048
15303,nm12444764,0.000048
15302,nm12015373,0.000048
15301,nm10843335,0.000048


In [18]:
#Setting the actor we will be comparing to
# target_actor_id = 'nm1165110' # Chris Hemsworth
# target_actor_id = 'nm0413168' # Hugh Jackman
# target_actor_id = 'nm0005351' # Ryan Reynolds
target_actor_id2 = "nm0000206" # Keanu Reeves
target_actor_id = 'nm0000115' # Nic Cage

In [19]:
per_cluster_prs = []
for cluster_id,group in cluster_df.groupby("cluster"):

    this_cluster_actor_count = sum([
        1 if target_actor_id in movie_actor_map[m]["actors"] else 0 
        for m in group["movie_id"]
    ])
    
    # Calculate conditional probability of the target actor given this cluster
    ## Pr(X=Actor | Y=Cluster <ID>)
    pr_actor_given_cluster = this_cluster_actor_count / group.shape[0]
    print("Pr[%s| Cluster %02d]:" % (target_actor_id, cluster_id), "\t", pr_actor_given_cluster)
    
    # Then add this calculated probability to `per_cluster_prs`,
    ## so we can track this conditional probability
    joint_pr_actor_cluster = pr_actor_given_cluster * group.shape[0] / cluster_df.shape[0]
    print("Pr[%s, Cluster %02d]:" % (target_actor_id, cluster_id), "\t", joint_pr_actor_cluster)
    per_cluster_prs.append(joint_pr_actor_cluster)

Pr[nm0000115| Cluster 00]: 	 0.0016144656118824668
Pr[nm0000115, Cluster 00]: 	 0.00024248302618816683
Pr[nm0000115| Cluster 01]: 	 0.005813953488372093
Pr[nm0000115, Cluster 01]: 	 0.0003879728419010669
Pr[nm0000115| Cluster 02]: 	 0.00665335994677312
Pr[nm0000115, Cluster 02]: 	 0.00048496605237633366
Pr[nm0000115| Cluster 03]: 	 0.004838709677419355
Pr[nm0000115, Cluster 03]: 	 0.0002909796314258002
Pr[nm0000115| Cluster 04]: 	 0.0015267175572519084
Pr[nm0000115, Cluster 04]: 	 4.8496605237633365e-05
Pr[nm0000115| Cluster 05]: 	 0.0
Pr[nm0000115, Cluster 05]: 	 0.0
Pr[nm0000115| Cluster 06]: 	 0.0006295247088448222
Pr[nm0000115, Cluster 06]: 	 9.699321047526673e-05
Pr[nm0000115| Cluster 07]: 	 0.0
Pr[nm0000115, Cluster 07]: 	 0.0
Pr[nm0000115| Cluster 08]: 	 0.0
Pr[nm0000115, Cluster 08]: 	 0.0
Pr[nm0000115| Cluster 09]: 	 0.0078125
Pr[nm0000115, Cluster 09]: 	 0.0001454898157129001
Pr[nm0000115| Cluster 10]: 	 0.0
Pr[nm0000115, Cluster 10]: 	 0.0
Pr[nm0000115| Cluster 11]: 	 0.0031

In [21]:
# For each cluster ID, calculate the posterior probability given the target actor
pr_target_actor = sum(per_cluster_prs)
for cluster_id,cluster_actor_pr in enumerate(per_cluster_prs):

    # combine this cluster_actor_pr, cluster_pr_df, and actor_prs_df
    ## to calculate Pr(Cluster|Actor)
    pr_cluster_given_actor = cluster_actor_pr / pr_target_actor

    print("Pr[Cluster %02d | %s]:" % (cluster_id, target_actor_id), "\t", pr_cluster_given_actor)
    

Pr[Cluster 00 | nm0000115]: 	 0.0819672131147541
Pr[Cluster 01 | nm0000115]: 	 0.13114754098360656
Pr[Cluster 02 | nm0000115]: 	 0.1639344262295082
Pr[Cluster 03 | nm0000115]: 	 0.09836065573770492
Pr[Cluster 04 | nm0000115]: 	 0.01639344262295082
Pr[Cluster 05 | nm0000115]: 	 0.0
Pr[Cluster 06 | nm0000115]: 	 0.03278688524590164
Pr[Cluster 07 | nm0000115]: 	 0.0
Pr[Cluster 08 | nm0000115]: 	 0.0
Pr[Cluster 09 | nm0000115]: 	 0.04918032786885246
Pr[Cluster 10 | nm0000115]: 	 0.0
Pr[Cluster 11 | nm0000115]: 	 0.03278688524590164
Pr[Cluster 12 | nm0000115]: 	 0.03278688524590164
Pr[Cluster 13 | nm0000115]: 	 0.06557377049180328
Pr[Cluster 14 | nm0000115]: 	 0.01639344262295082
Pr[Cluster 15 | nm0000115]: 	 0.27868852459016397
